In [446]:
import pandas as pd
import geopandas as gpd

In [447]:
excel = pd.read_excel("./resultats.xlsx", dtype = str)

In [448]:
geojson = gpd.read_file("./clean-contours.json")

In [449]:
paris =  gpd.read_file('./paris.json')

In [450]:
excel.at[7984, 'Code du b.vote'] = "40.1" #ajustement caen

In [451]:
excel['Code du b.vote'] = excel['Code du b.vote'].str.lstrip('0') #enlever les 0 pour correspondre au geojson

In [452]:
#tom + bureau étranger
excel = excel[excel['Code du département'] != 'ZZ']
excel = excel[excel['Code du département'] != 'ZX']
excel = excel[excel['Code du département'] != 'ZW']
excel = excel[excel['Code du département'] != 'ZP']
excel = excel[excel['Code du département'] != 'ZN']
excel.loc[~excel['Code du département'].astype(str).str.contains(r'\d', na=False), 'Code du département'] = '97'
#virer paris
excel = excel[excel['Libellé de la commune'] != 'Paris']

In [453]:
#renommer les unamed de excel
def renameShit():
    to_rename = ['N°Panneau', 'Sexe', 'Nom', 'Prénom', 'Voix', '% Voix/Ins','% Voix/Exp']
    for curr in to_rename:
        excel.rename(columns={curr: curr + '.1'}, inplace=True)
    unnamed = ['Unnamed: 28', 'Unnamed: 29','Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32','Unnamed: 33','Unnamed: 34']
    for i in range(len(to_rename)):
        excel.rename(columns={unnamed[i]: to_rename[i] + '.2'}, inplace=True)
renameShit()

In [454]:
#nouveau df formatted avec juste les colonnes importantes de excel
formatted = gpd.GeoDataFrame()
formatted['id_bvote'] = excel['Code du département'] + excel['Code de la commune'] + '_' + excel['Code du b.vote']
nom = ['Libellé de la commune', 'Votants', '% Abs/Ins', '% Voix/Exp.1', '% Voix/Exp.2']
for curr in nom:
    formatted[nom] = excel[nom]
formatted.rename(columns={'% Voix/Exp.1': '% macron_emmanuel'}, inplace=True)
formatted.rename(columns={'% Voix/Exp.2': '% le_pen_marine'}, inplace=True)

In [455]:
paris = paris.drop(columns=['type_election', 'annee', 'numero_tour', 'date_tour', 'circ_bv', 'quartier_bv', 'arr_bv', 
                    'sec_bv', 'num_bureau', 'nb_procuration', 'nb_emargement', 'nb_vote_blanc',
                    'nb_vote_nul', 'geo_point_2d', 'created_user', 'created_date', 'last_edited_user',
                   'last_edited_date', 'st_area_shape', 'st_perimeter_shape'], axis=1)

In [456]:
paris['% Abs/Ins'] = (paris['nb_inscrit'] - paris['nb_votant']) / paris['nb_inscrit'] * 100
paris['% macron_emmanuel'] = paris['macron_emmanuel'] / paris['nb_exprime'] * 100
paris['% le_pen_marine'] = paris['le_pen_marine'] / paris['nb_exprime'] * 100

In [457]:
paris = paris.drop(['nb_inscrit', 'nb_exprime', 'macron_emmanuel', 'le_pen_marine'], axis=1)

In [458]:
paris.rename(columns={'nb_votant':'Votants'}, inplace=True)
paris.rename(columns={'id_bvote':'id_bv'}, inplace=True)
formatted.rename(columns={'id_bvote':'id_bv'}, inplace=True)

In [459]:
paris['Libellé de la commune'] = 'Paris'

In [460]:
#virer paris de geojson
geojson = geojson[~geojson['id_bv'].str.startswith('75')]
paris_copy = paris

In [461]:
paris = paris.drop(['geometry'], axis=1)

In [462]:
formatted = pd.concat([formatted, paris], ignore_index=True)

In [463]:
paris_copy = paris_copy.drop(['% Abs/Ins', '% macron_emmanuel', '% le_pen_marine', 'Libellé de la commune', 'Votants'], axis=1)

In [464]:
geojson = pd.concat([geojson, paris_copy], ignore_index=True)

In [465]:
merged = geojson.merge(formatted, on="id_bv", how="left")
merged = gpd.GeoDataFrame(merged)

In [466]:
merged.to_file('contours.json')